In [344]:
import numpy as np
import pandas as pd
import ast # For changing to dictionary
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
import pickle

### You can download the data at https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

In [345]:
movies_df = pd.read_csv("C:/Users/ASUS/Documents/Workspace/System_Recommender/tmdb_5000_movies.csv")
credits_df = pd.read_csv("C:/Users/ASUS/Documents/Workspace/System_Recommender/tmdb_5000_credits.csv")

# EDA

In [346]:
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [347]:
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [348]:
# Rename ID
credits_df.rename(columns={'movie_id':'id'}, inplace=True)
credits_df.columns

Index(['id', 'title', 'cast', 'crew'], dtype='object')

In [349]:
# Inspect the data shape
print(credits_df.shape)
print(movies_df.shape)

(4803, 4)
(4803, 20)


In [350]:
# Inspect the data id Value
print(credits_df['id'].value_counts().shape)
print(movies_df['id'].value_counts().shape)

(4803,)
(4803,)


In [351]:
movies_merged = movies_df.merge(credits_df, on='id')

In [352]:
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [353]:
movies_merged.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [354]:
# Add year to title
movies_merged['year'] = movies_merged['release_date'].str.split('-').str[0]
movies_merged['title_x'] = movies_merged['title_x'] + ' (' + movies_merged['year'].values + ")"
movies_merged['title_x'].values

array(['Avatar (2009)', "Pirates of the Caribbean: At World's End (2007)",
       'Spectre (2015)', ..., 'Signed, Sealed, Delivered (2013)',
       'Shanghai Calling (2012)', 'My Date with Drew (2005)'],
      dtype=object)

## Feature Extraction

Untuk melakukan sistem rekomendasi, kita hanya memerlukan beberapa fitur seperti :
- genre
- id
- keywords
- title
- overview
- cast
- crew

In [355]:
important_columns = ['genres', 'id', 'keywords', 'title_x', 'overview', 'cast', 'crew']

movies_data = movies_merged[important_columns].rename(columns={"title_x":"title"})
movies_data.head(3)

,genres,id,keywords,title,overview,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar (2009),"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End (2007),"Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre (2015),A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [356]:
# Cek data kosong
movies_data.isnull().sum()

genres      0
id          0
keywords    0
title       1
overview    3
cast        0
crew        0
dtype: int64

In [357]:
# Cek data dengan overview
movies_data.loc[movies_data['overview'].isnull() == True]

,genres,id,keywords,title,overview,cast,crew
2656,"[{""id"": 18, ""name"": ""Drama""}]",370980,"[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...",Chiamatemi Francesco - Il Papa della gente (2015),NaN,"[{""cast_id"": 5, ""character"": ""Jorge Mario Berg...","[{""credit_id"": ""5660019ac3a36875f100252b"", ""de..."
4140,"[{""id"": 99, ""name"": ""Documentary""}]",459488,"[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...","To Be Frank, Sinatra at 100 (2015)",NaN,"[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."
4431,"[{""id"": 99, ""name"": ""Documentary""}]",292539,[],Food Chains (2014),NaN,[],"[{""credit_id"": ""5470c3b1c3a368085e000abd"", ""de..."


In [358]:
# Bukan film yang bagus, bisa kita hilangkan
movies_data.dropna(subset=['overview'], inplace=True)

In [359]:
print(movies_data.isnull().sum())
print(movies_data.shape)

genres      0
id          0
keywords    0
title       1
overview    0
cast        0
crew        0
dtype: int64
(4800, 7)


In [360]:
# Cek duplikasi
movies_data.duplicated().sum()

0

In [361]:
# Extract genre name, and remove space in word
print(movies_data['genres'][0])
movies_data['genres'] = movies_data['genres'].map(lambda x: [y['name'].replace(' ', '') for y in ast.literal_eval(x)])

[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]


In [362]:
# Extract keyword name, and remove space in word
print(movies_data['keywords'][0])
movies_data['keywords'] = movies_data['keywords'].map(lambda x: [y['name'].replace(' ', '') for y in ast.literal_eval(x)])

[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]


In [363]:
# Extract character name, and remove space in word
# print(movies_data['cast'][0])
movies_data['characters'] = movies_data['cast'].map(lambda x: [y['character'].replace(' ', '') for y in ast.literal_eval(x)][:5])

In [364]:
# Extract cast name, and remove space in word
# print(movies_data['cast'][0])
movies_data['cast'] = movies_data['cast'].map(lambda x: [y['character'].replace(' ', '') for y in ast.literal_eval(x)][:5])

In [365]:
# Extract crew name, and remove space in word
# print(movies_data['crew'][0])
movies_data['crew'] = movies_data['crew'].map(lambda x: [y['name'].replace(' ', '') for y in ast.literal_eval(x) if y['job'] == 'Director'])

In [366]:
# Change overview sentence to list
print(movies_data['overview'][0])
movies_data['overview'] = movies_data['overview'].map(lambda x: [y for y in x.split(' ')])

In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.


In [367]:
movies_data.head()

,genres,id,keywords,title,overview,cast,crew,characters
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar (2009),"[In, the, 22nd, century,, a, paraplegic, Marin...","[JakeSully, Neytiri, Dr.GraceAugustine, Col.Qu...",[JamesCameron],"[JakeSully, Neytiri, Dr.GraceAugustine, Col.Qu..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End (2007),"[Captain, Barbossa,, long, believed, to, be, d...","[CaptainJackSparrow, WillTurner, ElizabethSwan...",[GoreVerbinski],"[CaptainJackSparrow, WillTurner, ElizabethSwan..."
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre (2015),"[A, cryptic, message, from, Bond’s, past, send...","[JamesBond, Blofeld, Madeleine, M, Lucia]",[SamMendes],"[JamesBond, Blofeld, Madeleine, M, Lucia]"
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises (2012),"[Following, the, death, of, District, Attorney...","[BruceWayne/Batman, AlfredPennyworth, JamesGor...",[ChristopherNolan],"[BruceWayne/Batman, AlfredPennyworth, JamesGor..."
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",John Carter (2012),"[John, Carter, is, a, war-weary,, former, mili...","[JohnCarter, DejahThoris, Sola, TarsTarkas, Ta...",[AndrewStanton],"[JohnCarter, DejahThoris, Sola, TarsTarkas, Ta..."


In [368]:
# Create feature tags from genres, keywords, characters, cast, crew, and overview
movies_data['tags'] = movies_data['genres'] + movies_data['keywords'] + movies_data['characters'] + movies_data['cast'] + movies_data['crew'] + movies_data['overview']
movies_data.head()

,genres,id,keywords,title,overview,cast,crew,characters,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar (2009),"[In, the, 22nd, century,, a, paraplegic, Marin...","[JakeSully, Neytiri, Dr.GraceAugustine, Col.Qu...",[JamesCameron],"[JakeSully, Neytiri, Dr.GraceAugustine, Col.Qu...","[Action, Adventure, Fantasy, ScienceFiction, c..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End (2007),"[Captain, Barbossa,, long, believed, to, be, d...","[CaptainJackSparrow, WillTurner, ElizabethSwan...",[GoreVerbinski],"[CaptainJackSparrow, WillTurner, ElizabethSwan...","[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre (2015),"[A, cryptic, message, from, Bond’s, past, send...","[JamesBond, Blofeld, Madeleine, M, Lucia]",[SamMendes],"[JamesBond, Blofeld, Madeleine, M, Lucia]","[Action, Adventure, Crime, spy, basedonnovel, ..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises (2012),"[Following, the, death, of, District, Attorney...","[BruceWayne/Batman, AlfredPennyworth, JamesGor...",[ChristopherNolan],"[BruceWayne/Batman, AlfredPennyworth, JamesGor...","[Action, Crime, Drama, Thriller, dccomics, cri..."
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",John Carter (2012),"[John, Carter, is, a, war-weary,, former, mili...","[JohnCarter, DejahThoris, Sola, TarsTarkas, Ta...",[AndrewStanton],"[JohnCarter, DejahThoris, Sola, TarsTarkas, Ta...","[Action, Adventure, ScienceFiction, basedonnov..."


# Vectorization NLTK Stemmer & Count Vectorization

In [369]:
# Define Stemmer
stemmer = PorterStemmer()

# Stem the tags and join them into a lowered string
movies_data['tags'] = movies_data['tags'].map(lambda x: " ".join([stemmer.stem(y.lower()) for y in x]))
movies_data.head()

,genres,id,keywords,title,overview,cast,crew,characters,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar (2009),"[In, the, 22nd, century,, a, paraplegic, Marin...","[JakeSully, Neytiri, Dr.GraceAugustine, Col.Qu...",[JamesCameron],"[JakeSully, Neytiri, Dr.GraceAugustine, Col.Qu...",action adventur fantasi sciencefict culturecla...
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End (2007),"[Captain, Barbossa,, long, believed, to, be, d...","[CaptainJackSparrow, WillTurner, ElizabethSwan...",[GoreVerbinski],"[CaptainJackSparrow, WillTurner, ElizabethSwan...",adventur fantasi action ocean drugabus exotici...
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre (2015),"[A, cryptic, message, from, Bond’s, past, send...","[JamesBond, Blofeld, Madeleine, M, Lucia]",[SamMendes],"[JamesBond, Blofeld, Madeleine, M, Lucia]",action adventur crime spi basedonnovel secreta...
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises (2012),"[Following, the, death, of, District, Attorney...","[BruceWayne/Batman, AlfredPennyworth, JamesGor...",[ChristopherNolan],"[BruceWayne/Batman, AlfredPennyworth, JamesGor...",action crime drama thriller dccomic crimefight...
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",John Carter (2012),"[John, Carter, is, a, war-weary,, former, mili...","[JohnCarter, DejahThoris, Sola, TarsTarkas, Ta...",[AndrewStanton],"[JohnCarter, DejahThoris, Sola, TarsTarkas, Ta...",action adventur sciencefict basedonnovel mar m...


In [370]:
# Create new dataset only contain id, title, and tags
movies_data_final = movies_data[['id', 'title', 'tags']]

movies_data_final.head()

,id,title,tags
0,19995,Avatar (2009),action adventur fantasi sciencefict culturecla...
1,285,Pirates of the Caribbean: At World's End (2007),adventur fantasi action ocean drugabus exotici...
2,206647,Spectre (2015),action adventur crime spi basedonnovel secreta...
3,49026,The Dark Knight Rises (2012),action crime drama thriller dccomic crimefight...
4,49529,John Carter (2012),action adventur sciencefict basedonnovel mar m...


In [371]:
# Define Count Vectorizer
count_vector = CountVectorizer(stop_words='english', max_features=5000)

# fit count vectorizer on tags
vectors = count_vector.fit_transform(movies_data_final['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [372]:
count_vector.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zorro'], dtype=object)

In [373]:
similarity_matrix = cosine_similarity(vectors)
similarity_matrix

array([[1.        , 0.06976744, 0.0742156 , ..., 0.04075696, 0.        ,
        0.        ],
       [0.06976744, 1.        , 0.04947707, ..., 0.02037848, 0.        ,
        0.02381628],
       [0.0742156 , 0.04947707, 1.        , ..., 0.02167775, 0.        ,
        0.        ],
       ...,
       [0.04075696, 0.02037848, 0.02167775, ..., 1.        , 0.03367175,
        0.04173919],
       [0.        , 0.        , 0.        , ..., 0.03367175, 1.        ,
        0.07870418],
       [0.        , 0.02381628, 0.        , ..., 0.04173919, 0.07870418,
        1.        ]])

In [374]:
similarity_matrix.shape

(4800, 4800)

In [375]:
similarity_matrix[1]

array([0.06976744, 1.        , 0.04947707, ..., 0.02037848, 0.        ,
       0.02381628])

In [377]:
def recommend(movie):
    # Get movie index
    movie_index = movies_data_final[movies_data_final['title'] == movie].index[0]
    indices = pd.Series(similarity_matrix[movie_index].argsort()[-10:-1][::-1])

    # Get the title of the 10 most similar movies
    result = movies_data_final['title'].iloc[indices].values
    print(result)

recommend("Avatar (2009)")

['Lifeforce (1985)' 'The Thing (1982)'
 'Escape from the Planet of the Apes (1971)'
 '2001: A Space Odyssey (1968)'
 'Jekyll and Hyde ... Together Again (1982)' 'Lost in Space (1998)'
 'Capricorn One (1977)' 'Sphere (1998)' 'Dolphin Tale 2 (2014)']


# Export the DataFrame in dictionary and the similarity Data
export to .pkl using libraryi pickle

In [378]:
# Export final data to pkl
pickle.dump(movies_data_final.to_dict(), open('C:/Users/ASUS/Documents/Workspace/System_Recommender/movies_data.pkl', 'wb'))
pickle.dump(similarity_matrix, open('C:/Users/ASUS/Documents/Workspace/System_Recommender/similarity_matrix.pkl', 'wb'))